# TODO:
* Are the outputs the actual labels or just the loss (and if so, how do we specify the loss type)
* Turn data into tensor prior to making datasets
* Do we need to add beginning / end tokens to each text sample?

In [99]:
import pandas as pd
import numpy as np
import torch
from pytorch_pretrained_bert import BertForTokenClassification

from transformers import BertTokenizer
from torch.optim import SGD
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader, TensorDataset

from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm
from IPython.display import display 

import random

In [2]:
seed = 31

## Set the random seeds for Python and Torch
random.seed(seed)
np.random.seed(seed)

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.cuda.manual_seed(seed)

def seed_worker(worker_id): #function to initalize the seeds for the workers of DataLoader
    worker_seed = torch.initial_seed() %2 ** 32
    np.random.seed(worker_seed)
    random.seed(worker_seed)

g_seed = torch.Generator()
g_seed.manual_seed(seed)

In [48]:
#Load data from CSV, remove index column and separate into training and test sets
df = pd.read_csv('data/cleandata.csv')
df.dropna(inplace = True)

X = df['text']
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [101]:
def tokenize(row,my_token):
    try:
       return torch.tensor(my_token.encode_plus(row)['input_ids'])
    except:
        with open('bad_file.txt','a') as fid:
            fid.write(str(row.name) + '\n')
            

In [102]:
#Instantiate BERT Tokenizer and transform strings to tensors

my_token = BertTokenizer.from_pretrained("bert-base-cased")
X_train_token = X_train.apply(lambda x : tokenize(x,my_token))
X_test_token = X_test.apply(lambda x : tokenize(x,my_token))

Token indices sequence length is longer than the specified maximum sequence length for this model (655 > 512). Running this sequence through the model will result in indexing errors


In [56]:
y_test = torch.tensor(y_test.values) 
y_train = torch.tensor(y_train.values)   

In [55]:
#Define Hyperparameters
batch_size = 128
lr = 0.01

In [10]:
test_data = TensorDataset(X_test,y_test)
train_data = TensorDataset(X_train, y_train)

test_loader = DataLoader(test_data,batch_size = batch_size , shuffle = False, num_workers = 0, worker_init_fn = seed_worker, generator = g_seed)
train_loader = DataLoader(train_data,batch_size = batch_size , drop_last = True, shuffle = True, worker_init_fn = seed_worker, generator = g_seed)

TypeError: 'int' object is not callable

In [19]:
##Loading a pre-trained BERT model
model = BertForTokenClassification.from_pretrained('bert-base-uncased',
                                                   num_labels = 2)

In [ ]:
my_optim = SGD(model.parameters(), lr=lr)
my_loss = CrossEntropyLoss() #we might not need this, since the link in the word docuemt says that running the model returns a loss

model.train() #set the model to train mode


In [ ]:
my_optim.zero_grad()

y_pred,something_else = model(X_train)

my_loss(y_pred,y_train)

my_loss.backward()
my_optim.step()

